In [4]:
!pip install streamlit pyngrok langchain_groq groq langchain-core --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 3.8 MB/s eta 0:00:00


In [9]:
from pyngrok import ngrok, conf

conf.get_default().auth_token = "2zBH9P56i9oTDeTX7TOb2cawSio_YEXGXS5y8VZPg2fTQwPi"


In [23]:
%%writefile app.py
import streamlit as st
import pandas as pd
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.exceptions import OutputParserException
from langchain.chat_models import init_chat_model

# Initialize model only once
model = init_chat_model("llama3-8b-8192", model_provider="groq", api_key="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", temperature=0.2)
# Define the prompt
def extract(article_text):
  prompt = """From the below article, extract revenue and eps in JSON format containing the following keys:
    'revenue_actual','revenue_expected','eps_actual','eps_expected'
    Only return the valid JSON, no preamble.
    Article: {article}
    """
  pt = PromptTemplate.from_template(prompt)
  global model
  chain = pt | model

  result = chain.invoke({"article": article_text})
  parser = JsonOutputParser()
  output_json = parser.parse(result.content)
  return output_json

# Streamlit UI
st.title("Financial Data Extractor")

paragraph = st.text_area("Enter financial report paragraph:")

if st.button("Extract"):
    result = extract(paragraph)

    data = {
        "Measure": ["Revenue", "EPS"],
        "Estimated": [result['revenue_expected'], result['eps_expected']],
        "Actual": [result['revenue_actual'], result['eps_actual']]
    }

    df = pd.DataFrame(data)
    st.table(df)


Overwriting app.py


In [24]:
public_url = ngrok.connect(8501)
print("🌐 Streamlit app URL:", public_url)

!streamlit run app.py --server.port 8501 --server.enableCORS false


🌐 Streamlit app URL: NgrokTunnel: "https://7ed1-104-196-151-180.ngrok-free.app" -> "http://localhost:8501"
2025-06-29 13:05:24.236 
'server.enableXsrfProtection=true'.
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://104.196.151.180:8501

  Stopping...
^C
